<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamC/blob/main/Code/Datenbank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kursarbeit Datenmanagement und -Archivierung WS 23/24**

Master-Code Name-CALICO:MA


**Erstellung der Datenbank**


*   Datenbankinitialisierung:



> Importing Libraries

In [ ]:
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv
import requests

> Verbindung zu Googledrive herstellen (Ablageort der CSV-Dateien)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
material_path = "/content/drive/My Drive/csv_data"

In [ ]:
!ls "/content/drive/My Drive/csv_data"

> Verbindung zu SQLite herstellen und Datebankinitialisierung im Memory


In [ ]:
# SQLite connection
# Create a SQLite database in memory from python

def create_connection_memory():
    """ create a database connection to a database that resides
        in the memory
    """
    conn = None;
    try:
       #Establishing the connection
        conn = sqlite3.connect(':memory:')
        return conn
        print(sqlite3.version)
    except Error as e:
        print("Error while connecting to sqlite", e)
    # finally:
    #     if conn:
    #         conn.close()

In [ ]:
# Creating a temporary empty database in the memory.
conn = create_connection_memory()
# Creating a cursor object using the cursor() method
cur = conn.cursor()
print("Successfully Connected to SQLite")




> Verbindung zu SQLite herstellen und Datebankinitialisierung Local




In [ ]:
def create_connection_local(local_path):
  conn = None;
  try:
    #Establishing the connection
    conn = sqlite3.connect(local_path+'/datawarehouse.db')
    return conn
    print(sqlite3.version)
  except Error as e:
    print("Error while connecting to sqlite", e)
conn = create_connection_local('/content/drive/My Drive/Datenmanagement_und_Archivierung_im_Umfeld_der_Forschung/CALICO_MA')
# Creating a cursor object using the cursor() method
cur = conn.cursor()
print("Successfully Connected to SQLite Public Data Warehouse")


> Quelldatenimport

In [ ]:
# Load cancer CSVs
procedures = pd.read_csv('/content/drive/My Drive/Data_source/procedures.csv', sep=",")
encounters = pd.read_csv('/content/drive/My Drive/Data_source/encounters.csv', sep=",")
immunizations = pd.read_csv('/content/drive/My Drive/Data_source/immunizations.csv', sep=",")
medications = pd.read_csv('/content/drive/My Drive/Data_source/medications.csv', sep=",")
observations = pd.read_csv('/content/drive/My Drive/Data_source/observations.csv', sep=",")
patients = pd.read_csv('/content/drive/My Drive/Data_source/patients.csv', sep=",")

In [ ]:
#Delete Tables in case they exist
cur.execute("DROP TABLE IF EXISTS procedures")
cur.execute("DROP TABLE IF EXISTS encounters")
cur.execute("DROP TABLE IF EXISTS immunizations")
cur.execute("DROP TABLE IF EXISTS medications")
cur.execute("DROP TABLE IF EXISTS observations")
cur.execute("DROP TABLE IF EXISTS patients")
cur.execute("DROP TABLE IF EXISTS facts_table")

In [ ]:
# Create tables including facts_table
sql_create_source_data = requests.get('https://raw.githubusercontent.com/Fuenfgeld/DMA2024TeamC/main/Code/create_statements_db.sql').text
cur.executescript(sql_create_source_data)
conn.commit()
print("Successfully created tables in the database")

In [ ]:
#Insert data into tables

procedures.to_sql('procedures', conn, if_exists='append', index=False)
encounters.to_sql('encounters', conn, if_exists='append', index=False)
immunizations.to_sql('immunizations', conn, if_exists='append', index=False)
medications.to_sql('medications', conn, if_exists='append', index=False)
observations.to_sql('observations', conn, if_exists='append', index=False)
patients.to_sql('patients', conn, if_exists='append', index=False)


> Daten in Faktentabelle übertragen



In [ ]:
# Need to include date
cur.execute('''INSERT INTO facts_table
                    ( cancer_type, patient_ID, observation_code, observations_VALUE)
                    SELECT Table_Names, PATIENT, CODE, VALUE
                    FROM observations
                    ;''')


cur.execute('''INSERT INTO facts_table
                    (cancer_type, patient_ID, encounter_ID, procedure_code , procedures_BASE_COST )
                    SELECT Table_Names, PATIENT, ENCOUNTER, CODE, BASE_COST
                    FROM procedures
                    ;''')


cur.execute('''INSERT INTO facts_table
                    (cancer_type, patient_ID, encounter_ID,  medication_code, medications_TOTALCOST )
                    SELECT Table_Names, PATIENT, ENCOUNTER, CODE, TOTALCOST
                    FROM medications
                    ;''')

cur.execute('''INSERT INTO facts_table
                    (cancer_type, patient_ID, encounter_ID, immunization_code)
                    SELECT Table_Names, PATIENT,  ENCOUNTER, CODE
                    FROM immunizations
                    ;''')

cur.execute('''INSERT INTO facts_table
                    (cancer_type, patient_ID, encounter_ID, encounter_BASE_ENCOUNTER_COST)
                    SELECT Table_Names, PATIENT, Id, BASE_ENCOUNTER_COST
                    FROM encounters
                    ;''')



In [ ]:
conn.commit()

In [ ]:
#connect to DB
conn = sqlite3.connect('/content/drive/My Drive/Datenmanagement_und_Archivierung_im_Umfeld_der_Forschung/CALICO_MA/datawarehouse.db')
cursor = conn.cursor()


In [ ]:
#Select data
# show table: retrieve all the column of each records and 4 rows
cur.execute("SELECT * FROM patients")
records = cur.fetchall()
# show 4 rows
for row in records[:4]:
  print(row)

In [ ]:
# List of columns in a table
cur.execute ("select * from facts_table")
col_names = cur.description
for row in col_names:
  print(row[0])
